In [1]:
# -*- coding: utf-8 -*-

'''
@Author   :   Corley Tang
@contact  :   cutercorleytd@gmail.com
@Github   :   https://github.com/corleytd
@Time     :   2023-12-05 7:50
@Project  :   Hands-on Crawler with Python-redis_with_python
Python操作Redis
'''

# 导入所需的库
import redis

# 1.基本使用

In [2]:
# 连接Redis：通过连接池方式连接
conn = redis.StrictRedis(  # StrictRedis用于实现大部分官方命令，Redis是StrictRedis的子类，用于向后兼容旧版本
    connection_pool=redis.ConnectionPool(
        host='127.0.0.1',
        port=6379,
        db=0,
        encoding='utf-8',
        decode_responses=True  # 设置解码
    )
)
conn

Redis<ConnectionPool<Connection<host=127.0.0.1,port=6379,db=0>>>

In [3]:
conn.set('code', '052056')

True

In [4]:
conn.get('code')

'052056'

# 2.使用pipeline
在需要多次操作Redis时，可以通过pipeline将多条Redis命令缓存起来，并一起发送给Redis去处理，从而减少网络IO，提升速度的效果。

In [5]:
pipe = conn.pipeline()
pipe

Pipeline<ConnectionPool<Connection<host=127.0.0.1,port=6379,db=0>>>

In [6]:
pipe.hset('language', 'Python', 1)

Pipeline<ConnectionPool<Connection<host=127.0.0.1,port=6379,db=0>>>

In [7]:
pipe.hset('language', 'Java', 2)

Pipeline<ConnectionPool<Connection<host=127.0.0.1,port=6379,db=0>>>

In [8]:
pipe.hset('language', 'C++', 3)

Pipeline<ConnectionPool<Connection<host=127.0.0.1,port=6379,db=0>>>

In [9]:
pipe.execute()

[0, 0, 0]

In [10]:
pipe.close()

In [11]:
# 使用with关键字操作pipeline
key = 'bigdata'
bigdata = ['Hadoop', 'Spark', 'Flink']
with conn.pipeline() as pipe:
    for score, item in enumerate(bigdata):
        pipe.zadd(key, {item: score})
    pipe.execute()  # 多个命令一起发给Redis

conn.zrange(key, 0, -1)

['Hadoop', 'Spark', 'Flink']

# 3.分布式锁
分布式锁的目的是为了保证同一时间只能有一个客户端对共享资源进行操作，其原理就是获取单个实例上的资源，从而实现锁住其他实例的效果。

In [12]:
# redis 分布式锁
def get_lock(key, timeout=10):
    lock = conn.lock(
        key,  # 锁名
        timeout=timeout,  # 阻塞超时时间，即当多个实例争夺锁时实例等待锁的最长时间
        blocking_timeout=timeout  # 锁的超时时间
    )
    return lock


key = 'lock'
with get_lock(key, timeout=30) as lock:
    if lock:
        print('Acquire lock successfully')
    else:
        print('Acquire lock failed')

Acquire lock successfully


In [13]:
# 关闭连接
conn.close()